# Import librairies

In [3]:
import os

seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
import sys
import random

sys.path.append("../utile")
sys.path.append("..")
import random
from database import create_cnx, config_parse
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

path = "../data"
plt.rcParams["figure.figsize"] = (15, 15)


random.seed(seed)
np.random.seed(seed)

# Loading data and utile functions

## Loading data from database

In [14]:
# Select all States, Districts, Sub_Districts, Blocks, GPs in the raw dataset to link it to the prediction file
parser = config_parse()
cnx = create_cnx(parser)["cnx"]
curs = cnx.cursor()
season = "rabi"
sql_request = f"""SELECT DISTINCT State, District, Sub_District, Block, GP from data_SCOR WHERE Season = "{season}" ORDER BY RAND() """

curs.execute(sql_request)
names = [x[0] for x in curs.description]
rows = curs.fetchall()
df = pd.DataFrame(rows, columns=names)
# df.drop("id", inplace=True, axis=1)
# df.dropna(axis=0, how="all", inplace=True)

# Display no matchs rate between raw dataset and prediction file

In [15]:
# Get all unique states, districts and blocks for raw dataset
state = df["State"].unique().tolist()
district = df["District"].unique().tolist()
sub_district = df["Sub_District"].unique().tolist()
block = df["Block"].unique().tolist()

In [16]:
# Load prediction file for season
path = os.path.join("..", f"03_Prediction", f"GP_Pred_{season.title()}_ID.csv")
df_to_predict = pd.read_csv(path)

In [17]:
# Get all unique states, districts and blocks for prediction file
df_to_predict.rename(columns={"Sub-District": "Sub_District"}, inplace=True)
pred_state = df_to_predict["State"].unique().tolist()
pred_block = df_to_predict["Block"].unique().tolist()
pred_district = df_to_predict["District"].unique().tolist()
pred_sub_district = df_to_predict["Sub_District"].unique().tolist()

In [18]:
print(
    f"Rate of States in pred files and not in raw dataset {int(100*len(set(pred_state)-set(state))/len(set(pred_state)))}%"
)
print()
print(
    f"Rate of Districts in pred files and not in raw dataset {int(100*len(set(pred_district)-set(district))/len(set(pred_district)))}%"
)
print()
print(
    f"Rate of Sub_Districts in pred files and not in raw dataset {int(100*len(set(pred_sub_district)-set(sub_district))/len(set(pred_sub_district)))}%"
)
print()
print(
    f"Rate of block in pred files and not in raw dataset {int(100*len(set(pred_block)-set(block))/len(set(pred_block)))}%"
)
print()

Rate of States in pred files and not in raw dataset 6%

Rate of Districts in pred files and not in raw dataset 7%

Rate of Sub_Districts in pred files and not in raw dataset 35%

Rate of block in pred files and not in raw dataset 60%



In [19]:
# Create a unique string with States, Districts,Sub_Districts and Blocks
def unique_string(x):
    d = [x["State"], x["District"], x["Sub_District"], x["Block"]]
    L = ["None" if elem in [None, np.NAN] else str(elem) for elem in d]
    return "/sep/".join(L)


df["loc_string"] = df.apply(unique_string, axis=1)
df_to_predict["loc_string"] = df_to_predict.apply(unique_string, axis=1)

In [20]:
# Tests but not efficient enough to be run
# Use get_close_matches for fuzzy match
# from difflib import get_close_matches
# COUNT = 0

# def increment():
#     global COUNT
#     COUNT = COUNT+1

# def get_match(x):
#     increment()
#     if COUNT%1000==0:
#         print(COUNT)
#     if not dfright[dfright["loc_string"]==x].empty:
#         return x
#     else:
#         try:
#             return get_close_matches(x,dfright["loc_string"])[0]
#         except:
#             return
# dfleft = pd.DataFrame(df_to_predict["loc_string"], columns = ["loc_string"]).drop_duplicates(ignore_index=False)
# dfright = pd.DataFrame(df["loc_string"], columns = ["loc_string"]).drop_duplicates(ignore_index=False)
# dfleft["loc_string2"] = dfleft["loc_string"].apply(get_match)

In [21]:
# Here we use a fuzzy match to get all values that are in the prediction file
from fuzzymatcher import link_table, fuzzy_left_join

left_on = ["loc_string"]
right_on = ["loc_string"]
dfleft = pd.DataFrame(
    df_to_predict["loc_string"], columns=["loc_string"]
).drop_duplicates(ignore_index=False)
dfright = pd.DataFrame(df["loc_string"], columns=["loc_string"]).drop_duplicates(
    ignore_index=False
)
fuzzy_match = fuzzy_left_join(dfleft, dfright, left_on, right_on)

In [22]:
fuzzy_match

,best_match_score,__id_left,__id_right,loc_string_left,loc_string_right
0,0.813499,0_left,10697_right,andhra pradesh/sep/anantapur/sep/vidapanakal/s...,andhra pradesh/sep/anantapur/sep/vidapanakal/s...
1,0.835273,1_left,7205_right,andhra pradesh/sep/anantapur/sep/vajrakarur/se...,andhra pradesh/sep/anantapur/sep/vajrakarur/se...
2,0.750306,2_left,14130_right,andhra pradesh/sep/anantapur/sep/gooty/sep/None,andhra pradesh/sep/anantapur/sep/gooty/sep/None
3,0.800559,3_left,3634_right,andhra pradesh/sep/anantapur/sep/guntakal/sep/...,andhra pradesh/sep/anantapur/sep/guntakal/sep/...
4,0.813499,4_left,5327_right,andhra pradesh/sep/anantapur/sep/pamidi/sep/None,andhra pradesh/sep/anantapur/sep/pamidi/sep/None
...,...,...,...,...,...
1014669,0.283099,45165_left,2405_right,uttar pradesh/sep/agra/sep/khandauli/sep/khanda,uttar pradesh/sep/agra/sep/eatmadpur/sep/khandoli
1014687,0.468471,45166_left,2405_right,uttar pradesh/sep/agra/sep/khandauli/sep/khand...,uttar pradesh/sep/agra/sep/eatmadpur/sep/khandoli
1014705,0.313202,45167_left,2405_right,uttar pradesh/sep/agra/sep/khandauli/sep/semra,uttar pradesh/sep/agra/sep/eatmadpur/sep/khandoli
1014723,0.306133,45168_left,3147_right,uttar pradesh/sep/agra/sep/shamsabad/sep/chitaura,uttar pradesh/sep/agra/sep/None/sep/shamshabad


In [23]:
# Export to be reused
fuzzy_match.to_csv(f"../material/files/fuzzy_match_{season}.csv")